# 4.0 Feature engineering
Use our existing features to develop new features, convert categorical values to one-hot vectors, and drop others. We will "fit" to the training data and then transform to the test data

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
   
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
DATA_FOLDER = os.path.join('data', 'processed_2')
SAVE_FOLDER = os.path.join('data', 'final')

### Read in our test and training data

In [3]:
zoopla_df_train = pd.read_csv(os.path.join(DATA_FOLDER, 'zoopla_train.csv'), dtype=str)
zoopla_df_test = pd.read_csv(os.path.join(DATA_FOLDER, 'zoopla_test.csv'), dtype=str)

In [4]:
zoopla_df_train.columns

Index(['listing_id', 'parish', 'post_town', 'postcode', 'latitude',
       'longitude', 'num_bedrooms', 'num_bathrooms', 'price',
       'property_type_general', 'CURRENT_ENERGY_RATING_mode',
       'POTENTIAL_ENERGY_RATING_mode', 'TOTAL_FLOOR_AREA_median',
       'NUMBER_HABITABLE_ROOMS_mode', 'CONSTRUCTION_AGE_BAND_mode',
       'Index of Multiple Deprivation Decile', 'Income Decile',
       'Employment Decile', 'Education and Skills Decile',
       'Health and Disability Decile', 'Crime Decile',
       'Barriers to Housing and Services Decile', 'Living Environment Decile',
       'IDACI Decile', 'IDAOPI Decile', 'PROB_4BAND', 'diff_published_date',
       'last_published_year', 'last_published_month', 'first_published_year',
       'first_published_month'],
      dtype='object')

In [5]:
zoopla_df_train.head()

,listing_id,parish,post_town,postcode,latitude,longitude,num_bedrooms,num_bathrooms,price,property_type_general,CURRENT_ENERGY_RATING_mode,POTENTIAL_ENERGY_RATING_mode,TOTAL_FLOOR_AREA_median,NUMBER_HABITABLE_ROOMS_mode,CONSTRUCTION_AGE_BAND_mode,Index of Multiple Deprivation Decile,Income Decile,Employment Decile,Education and Skills Decile,Health and Disability Decile,Crime Decile,Barriers to Housing and Services Decile,Living Environment Decile,IDACI Decile,IDAOPI Decile,PROB_4BAND,diff_published_date,last_published_year,last_published_month,first_published_year,first_published_month
0,63843030,"Nuneaton and Bedworth, unparished area",Nuneaton,CV11 6,52.52679,-1.435227,4,2,400000.0,Detached house,2.0,1.0,96.0,NaN,12.0,8,9,8,9,7,6,3,6,9,9,None,2,2023,4,2023,1
1,64031802,"Nuneaton and Bedworth, unparished area",Nuneaton,CV11 6,52.531292,-1.441875,2,1,280000.0,Bungalow,4.0,3.0,78.0,4.0,4.0,9,9,8,9,5,9,9,9,9,10,None,1,2023,4,2023,2
2,62869062,"Nuneaton and Bedworth, unparished area",Nuneaton,CV11 6,52.510036,-1.435902,3,2,300000.0,Detached house,4.0,2.0,107.0,6.0,5.0,10,10,9,9,8,9,6,10,9,9,None,3,2023,3,2022,11
3,64390832,"Nuneaton and Bedworth, unparished area",Nuneaton,CV11 6,52.505596,-1.42432,3,1,329950.0,Detached house,4.0,2.0,107.0,5.0,6.0,10,10,9,9,7,9,5,10,9,10,None,0,2023,4,2023,4
4,62804722,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0,52.550716,-1.371109,4,3,500000.0,Detached house,4.0,3.0,137.0,6.0,3.0,5,5,4,3,5,4,7,7,5,5,None,2,2023,1,2022,11


### Drop house prices greater than 1 million pounds and set target as log
Also remove 0 or negative house prices to ensure log does not blow up

In [6]:
zoopla_df_train = zoopla_df_train[(zoopla_df_train['price'].astype(float) <= 1000000) & (zoopla_df_train['price'].astype(float) >= 0)].reset_index()
zoopla_df_test = zoopla_df_test[(zoopla_df_test['price'].astype(float) <= 1000000) & (zoopla_df_test['price'].astype(float) >= 0)].reset_index()

In [7]:
y_train = np.log(zoopla_df_train['price'].astype(float)).rename('LOG_price')
y_test = np.log(zoopla_df_test['price'].astype(float)).rename('LOG_price')

### Separate out features into numeric (with a further separation for numeric vars we want converted to KMeans clusters), ordered discrete and categoric variables

In [8]:
numeric_cols = ['TOTAL_FLOOR_AREA_median', 'diff_published_date']

numeric_kmeans_cols = ['latitude', 'longitude']

discrete_cols = ['num_bedrooms', 'num_bathrooms', 'CURRENT_ENERGY_RATING_mode',
       'POTENTIAL_ENERGY_RATING_mode', 'CONSTRUCTION_AGE_BAND_mode',
       'Index of Multiple Deprivation Decile', 'Income Decile',
       'Employment Decile', 'Education and Skills Decile',
       'Health and Disability Decile', 'Crime Decile',
       'Barriers to Housing and Services Decile', 'Living Environment Decile',
       'IDACI Decile', 'IDAOPI Decile', 'last_published_year', 'last_published_month', 
        'first_published_year', 'first_published_month']

categoric_cols = ['post_town', 'parish', 'postcode', 'PROB_4BAND', 'property_type_general']

### Transform the four types of variables separately in a pipeline
Set up pipelines. For null values, we use the median in the training data for numeric fields and the mode for discrete and categoric fields.

In [9]:
class KMeansCluster(BaseEstimator, TransformerMixin):
    
    """
    Apply KMeans clustering to longitude and latitude. Want cluster centres fit to training
    data and the test data to be transformed based on the fits
    """
    
    def __init__(self, n_clusters=8):
        self.n_clusters = n_clusters        
            
    def fit(self, X, y=None):
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=40)
        self.kmeans.fit(X)
        return self
    
    def transform(self, X):
        cluster_centres = self.kmeans.transform(X).argmin(axis=1)
        X = cluster_centres.reshape(cluster_centres.shape[0], 1) # converts 1D array to 2D#
        X = pd.DataFrame(X, columns=['cluster'])
        self.X = X
        return X
        
        # return original co-ordinates and cluster number
        #print(np.c_[X, cluster_centres])
        #return np.c_[X, cluster_centres]
        
    def get_feature_names_out(self, X):
        return self.X.columns.tolist()


In [10]:
numeric_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median', add_indicator=False)),
    ('num_std_scaler', StandardScaler()),
])

numeric_kmeans_pipeline = Pipeline([
    ('num_kmeans_imputer', SimpleImputer(strategy='median', add_indicator=False)),
    ('num_kmeans_cluster', KMeansCluster(8)),
    ('num_kmeans_oh_encoder', OneHotEncoder(drop='first'))
])

discrete_pipeline = Pipeline([
    ('dis_imputer', SimpleImputer(strategy='most_frequent', add_indicator=False)),
    ('dis_std_scaler', StandardScaler()),
])

categoric_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent', add_indicator=False, )),
    ('cat_oh_encoder', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist'))
])

Set up a column transformer to handle all four types of columns together

In [11]:
full_pipeline = ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_cols),
    ('numeric_kmeans', numeric_kmeans_pipeline, numeric_kmeans_cols),
    ('discrete', discrete_pipeline, discrete_cols),
    ('categoric', categoric_pipeline, categoric_cols),
])

In [12]:
full_pipeline

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('num_imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('num_std_scaler',
                                                  StandardScaler())]),
                                 ['TOTAL_FLOOR_AREA_median',
                                  'diff_published_date']),
                                ('numeric_kmeans',
                                 Pipeline(steps=[('num_kmeans_imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('num_kmeans_cluster',
                                                  KMeansCluster()),
                                                 ('num_kmeans_oh_encoder'...
                                  'IDAOPI Decile', 'last_published_year',
                                  'last_published_month',
                                  'first_published_year',
                                  'first_published_month']),
                                ('categoric',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_oh_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['post_town', 'parish', 'postcode',
                                  'PROB_4BAND', 'property_type_general'])])

Fit to training data and transform it. Then apply those fitted values to the test data.

In [13]:
X_train = full_pipeline.fit_transform(zoopla_df_train)
X_test = full_pipeline.transform(zoopla_df_test, )

/home/robertv/anaconda3/envs/datascience2/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Check outputs

In [14]:
X_train[200]

array([ 0.66149868,  2.08416527,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  2.20174554,
        0.93059994,  1.64446487,  1.64628998, -0.55260864,  0.7718793 ,
        0.3262387 ,  1.37290084,  0.12961152,  1.61452849,  1.1674399 ,
       -1.06224429,  1.45224952,  0.06964659,  0.31241302, -2.56494588,
        2.10854468, -1.15042685, -0.62894756,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [15]:
full_pipeline.transformers_

[('numeric',
  Pipeline(steps=[('num_imputer', SimpleImputer(strategy='median')),
                  ('num_std_scaler', StandardScaler())]),
  ['TOTAL_FLOOR_AREA_median', 'diff_published_date']),
 ('numeric_kmeans',
  Pipeline(steps=[('num_kmeans_imputer', SimpleImputer(strategy='median')),
                  ('num_kmeans_cluster', KMeansCluster()),
                  ('num_kmeans_oh_encoder', OneHotEncoder(drop='first'))]),
  ['latitude', 'longitude']),
 ('discrete',
  Pipeline(steps=[('dis_imputer', SimpleImputer(strategy='most_frequent')),
                  ('dis_std_scaler', StandardScaler())]),
  ['num_bedrooms',
   'num_bathrooms',
   'CURRENT_ENERGY_RATING_mode',
   'POTENTIAL_ENERGY_RATING_mode',
   'CONSTRUCTION_AGE_BAND_mode',
   'Index of Multiple Deprivation Decile',
   'Income Decile',
   'Employment Decile',
   'Education and Skills Decile',
   'Health and Disability Decile',
   'Crime Decile',
   'Barriers to Housing and Services Decile',
   'Living Environment Decile',
   

Set the column names again

In [16]:
feature_names = full_pipeline.get_feature_names_out()

feature_names_new = [name.split('__')[1] for name in feature_names]
print(feature_names_new)

['TOTAL_FLOOR_AREA_median', 'diff_published_date', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'num_bedrooms', 'num_bathrooms', 'CURRENT_ENERGY_RATING_mode', 'POTENTIAL_ENERGY_RATING_mode', 'CONSTRUCTION_AGE_BAND_mode', 'Index of Multiple Deprivation Decile', 'Income Decile', 'Employment Decile', 'Education and Skills Decile', 'Health and Disability Decile', 'Crime Decile', 'Barriers to Housing and Services Decile', 'Living Environment Decile', 'IDACI Decile', 'IDAOPI Decile', 'last_published_year', 'last_published_month', 'first_published_year', 'first_published_month', 'post_town_Nuneaton', 'parish_Hinckley and Bosworth, unparished area', 'parish_Nuneaton and Bedworth, unparished area', 'parish_Other', 'parish_Stoke Golding', 'postcode_CV10 7', 'postcode_CV10 8', 'postcode_CV10 9', 'postcode_CV11 4', 'postcode_CV11 5', 'postcode_CV11 6', 'postcode_CV11 7', 'postcode_CV12 9', 'postcode_CV13 0', 'postcode_CV13 6', 'postcode_LE10 0', 'postc

In [17]:
X_train = pd.DataFrame(X_train, columns=feature_names_new)
y_train = pd.DataFrame(y_train, columns=['LOG_price'])

X_test = pd.DataFrame(X_test, columns=feature_names_new)
y_test = pd.DataFrame(y_test, columns=['LOG_price'])

### Save transformed train and test sets

In [18]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file_train = os.path.join(SAVE_FOLDER, f'zoopla_train.csv')
save_file_test = os.path.join(SAVE_FOLDER, f'zoopla_test.csv')

zoopla_df_train = pd.concat([X_train, y_train], axis=1)
zoopla_df_test = pd.concat([X_test, y_test], axis=1)

zoopla_df_train.to_csv(save_file_train, index=False)
zoopla_df_test.to_csv(save_file_test, index=False)

In [19]:
for colname in zoopla_df_train.columns:
    display(zoopla_df_train[colname].value_counts(dropna=False))

-0.208482    168
-0.444409     25
-0.237973     22
 0.115918     19
-0.650845     16
            ... 
-0.606609      1
 0.690990      1
-0.817468      1
 0.023464      1
-0.865391      1
Name: TOTAL_FLOOR_AREA_median, Length: 286, dtype: int64

-0.714820    442
-0.314965    179
 0.084890    120
 0.484745     78
 0.884600     50
 1.284455     50
 1.684310     31
 2.084165     24
 2.484020     13
 2.883875      8
 3.283731      4
 4.883151      3
 4.083441      3
 3.683586      1
 7.682136      1
 6.882426      1
Name: diff_published_date, dtype: int64

0.0    834
1.0    174
Name: cluster_1, dtype: int64

0.0    901
1.0    107
Name: cluster_2, dtype: int64

0.0    882
1.0    126
Name: cluster_3, dtype: int64

...

0.0    1000
1.0       8
Name: property_type_general_Maisonette, dtype: int64

0.0    979
1.0     29
Name: property_type_general_Other/Unknown, dtype: int64

0.0    730
1.0    278
Name: property_type_general_Semi-detached house, dtype: int64

0.0    834
1.0    174
Name: property_type_general_Terraced house, dtype: int64

12.429216    51
12.468437    34
12.691580    26
12.301383    26
12.254863    26
             ..
11.362103     1
13.764217     1
11.849040     1
13.592367     1
11.674194     1
Name: LOG_price, Length: 236, dtype: int64

In [20]:
zoopla_df_train

,TOTAL_FLOOR_AREA_median,diff_published_date,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,num_bedrooms,num_bathrooms,CURRENT_ENERGY_RATING_mode,POTENTIAL_ENERGY_RATING_mode,CONSTRUCTION_AGE_BAND_mode,Index of Multiple Deprivation Decile,Income Decile,Employment Decile,Education and Skills Decile,Health and Disability Decile,Crime Decile,Barriers to Housing and Services Decile,Living Environment Decile,IDACI Decile,IDAOPI Decile,last_published_year,last_published_month,first_published_year,first_published_month,post_town_Nuneaton,"parish_Hinckley and Bosworth, unparished area","parish_Nuneaton and Bedworth, unparished area",parish_Other,parish_Stoke Golding,postcode_CV10 7,postcode_CV10 8,postcode_CV10 9,postcode_CV11 4,postcode_CV11 5,postcode_CV11 6,postcode_CV11 7,postcode_CV12 9,postcode_CV13 0,postcode_CV13 6,postcode_LE10 0,postcode_LE10 1,postcode_LE10 2,postcode_LE10 3,postcode_LE9 7,postcode_LE9 8,postcode_LE9 9,PROB_4BAND_Medium,PROB_4BAND_None,property_type_general_Detached house,property_type_general_End terrace house,property_type_general_Flat,property_type_general_Maisonette,property_type_general_Other/Unknown,property_type_general_Semi-detached house,property_type_general_Terraced house,LOG_price
0,0.115918,0.084890,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.114890,0.930600,-1.887397,-1.965158,2.170512,0.771879,1.078753,0.962618,1.783389,0.721555,0.342209,-1.467828,-0.121802,1.266302,1.118847,0.389872,-0.020772,0.835241,-1.165536,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.899220
1,-0.414918,-0.314965,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.058822,-0.475764,0.467178,1.646290,-0.250040,1.149759,1.078753,0.962618,1.783389,-0.171419,1.580055,0.965677,1.058737,1.266302,1.522063,0.389872,-0.020772,0.835241,-0.897242,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.542545
2,0.440317,0.484745,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.028034,0.930600,0.467178,-0.159434,0.052529,1.527639,1.455010,1.372901,1.783389,1.168042,1.580055,-0.251076,1.452250,1.266302,1.118847,0.389872,-0.375658,-1.150427,1.517406,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.611538
3,0.440317,-0.714820,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.028034,-0.475764,0.467178,-0.159434,0.355098,1.527639,1.455010,1.372901,1.783389,0.721555,1.580055,-0.656660,1.452250,1.266302,1.522063,0.389872,-0.020772,0.835241,-0.360653,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.706696
4,1.325043,0.084890,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.114890,2.336964,0.467178,1.646290,-0.552609,-0.361760,-0.426275,-0.678513,-0.697277,-0.171419,-0.483022,0.154508,0.271711,-0.329238,-0.494021,0.389872,-1.085430,-1.150427,1.517406,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.122363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,-0.326445,-0.714820,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.114890,0.930600,-0.710110,-0.159434,1.565374,0.771879,0.326239,1.372901,0.129612,1.614528,1.167440,-1.062244,1.452250,0.069647,0.312413,-2.564946,2.108545,-1.150427,1.249112,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.506177
1004,-0.813045,-0.314965,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.058822,-0.475764,0.467178,-0.159434,-0.250040,1.527639,1.455010,0.962618,1.369945,0.721555,1.167440,0.965677,1.058737,1.266302,1.522063,0.389872,-0.730544,-1.150427,1.785700,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,